In [ ]:
#attempt with k-mer counting and CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df_class = pd.read_csv('/content/pdb_data_no_dups.csv')
df_seq = pd.read_csv('/content/pdb_data_seq.csv')

In [ ]:
protein_class = df_class[df_class.macromoleculeType == 'Protein']
protein_seq = df_seq[df_seq.macromoleculeType == 'Protein']

In [ ]:
#join the features
protein_class = protein_class[['structureId','classification']]
protein_seq = protein_seq[['structureId','sequence']]

In [ ]:
pdb_data = protein_class.set_index('structureId').join(protein_seq.set_index('structureId'))

In [ ]:
print('Generating the final dataset...')
pdb_data.head()

Generating the final dataset...


,classification,sequence
structureId,,
101M,OXYGEN TRANSPORT,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
102L,HYDROLASE(O-GLYCOSYL),MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
102M,OXYGEN TRANSPORT,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
103L,HYDROLASE(O-GLYCOSYL),MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...
103M,OXYGEN TRANSPORT,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...


In [ ]:
pdb_data.isnull().sum()

classification    1
sequence          3
dtype: int64

In [ ]:
pdb_data.dropna()

,classification,sequence
structureId,,
101M,OXYGEN TRANSPORT,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
102L,HYDROLASE(O-GLYCOSYL),MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
102M,OXYGEN TRANSPORT,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
103L,HYDROLASE(O-GLYCOSYL),MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...
103M,OXYGEN TRANSPORT,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
...,...,...
9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),MNYQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVQRLAEL...
9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...
9XIM,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),SVQATREDKFSFGLWTVGWQARDAFGDATRTALDPVEAVHKLAEIG...


In [ ]:
#no of classfications
counts = pdb_data.classification.value_counts()
print(counts)

HYDROLASE                                  46336
TRANSFERASE                                36424
OXIDOREDUCTASE                             34322
IMMUNE SYSTEM                              15615
LYASE                                      11682
                                           ...  
ANTIMICROBIAL PROTEIN, MEMBRANE PROTEIN        1
CALCIUM CHANNEL BLOCKER                        1
C-TYPE LECTIN                                  1
Cell adhesion, structural protein              1
Recombination/DNA binding                      1
Name: classification, Length: 4468, dtype: int64


In [ ]:
types = np.asarray(counts[(counts>1000)].index)
data = pdb_data[pdb_data.classification.isin(types)]
print(types)

['HYDROLASE' 'TRANSFERASE' 'OXIDOREDUCTASE' 'IMMUNE SYSTEM' 'LYASE'
 'HYDROLASE/HYDROLASE INHIBITOR' 'TRANSCRIPTION' 'VIRAL PROTEIN'
 'TRANSPORT PROTEIN' 'VIRUS' 'SIGNALING PROTEIN' 'ISOMERASE' 'LIGASE'
 'MEMBRANE PROTEIN' 'PROTEIN BINDING' 'STRUCTURAL PROTEIN' 'CHAPERONE'
 'STRUCTURAL GENOMICS, UNKNOWN FUNCTION' 'SUGAR BINDING PROTEIN'
 'DNA BINDING PROTEIN' 'PHOTOSYNTHESIS' 'ELECTRON TRANSPORT'
 'TRANSFERASE/TRANSFERASE INHIBITOR' 'METAL BINDING PROTEIN'
 'CELL ADHESION' 'UNKNOWN FUNCTION' 'PROTEIN TRANSPORT' 'TOXIN'
 'CELL CYCLE' 'RNA BINDING PROTEIN' 'DE NOVO PROTEIN' 'HORMONE'
 'GENE REGULATION' 'OXIDOREDUCTASE/OXIDOREDUCTASE INHIBITOR' 'APOPTOSIS'
 'MOTOR PROTEIN' 'PROTEIN FIBRIL' 'METAL TRANSPORT'
 'VIRAL PROTEIN/IMMUNE SYSTEM' 'CONTRACTILE PROTEIN' 'FLUORESCENT PROTEIN'
 'TRANSLATION' 'BIOSYNTHETIC PROTEIN']


In [ ]:
print(data)

                                classification                                           sequence
structureId                                                                                      
10GS         TRANSFERASE/TRANSFERASE INHIBITOR  PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...
10GS         TRANSFERASE/TRANSFERASE INHIBITOR  PPYTVVYFPVRGRCAALRMLLADQGQSWKEEVVTVETWQEGSLKAS...
117E                                 HYDROLASE  TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...
117E                                 HYDROLASE  TYTTRQIGAKNTLEYKVYIEKDGKPVSAFHDIPLYADKENNIFNMV...
11AS                                    LIGASE  MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...
...                                        ...                                                ...
9LPR             HYDROLASE/HYDROLASE INHIBITOR  ANIVGGIEYSINNASLCSVGFSVTRGATKGFVTAGHCGTVNATARI...
9LPR             HYDROLASE/HYDROLASE INHIBITOR                                              XAAPL
9NSE                

In [ ]:
data.dropna()
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(data['sequence'], data['classification'], test_size = 0.2, random_state = 1)

# Create a Count Vectorizer to gather the unique elements in sequence
vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (4,4))

# Fit and Transform CountVectorizer
vect.fit(X_train.values.astype('U'))
X_train_df = vect.transform(X_train.values.astype('U'))
X_test_df = vect.transform(X_test.values.astype('U'))

#Print a few of the features
print(vect.get_feature_names()[-20:])

['zhhh', 'ziar', 'zigi', 'ziwz', 'zkal', 'zkky', 'zknt', 'zkyh', 'zlik', 'zlzk', 'zpvm', 'zrgd', 'zrvi', 'ztvl', 'ztzk', 'zvbd', 'zvib', 'zvka', 'zwdl', 'zzvb']


In [ ]:
#training model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
pred_acc = dict()
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train_df,y_train)
pred = model.predict(X_test_df)
pred_acc['MultinomialNB'] = accuracy_score(pred,y_test)
print(pred_acc['MultinomialNB'])

0.7641194821960053
